<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/nearest_dc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopy
!pip install Nominatim
!pip install mpu

In [61]:
from geopy.geocoders import Nominatim
import os
import pandas as pd
import mpu
import numpy as np
from math import cos, asin, sqrt
from geopy import distance

In [26]:
df_sbd_dc = pd.read_csv("sbd_dc.txt")
# Initialize Nominatim API
geolocator = Nominatim(user_agent="test_sbd_dc")
country = "USA"
lat_list = []
long_list = []
for city, state in df_sbd_dc[['SHP_CITY_NAME', 'SHP_STATE']].values.tolist():
    loc = geolocator.geocode(city + ', ' + state + ', '+ country)
    lat, long = loc.latitude, loc.longitude
    lat_list.append(lat)
    long_list.append(long)
    print(city, state)
    print("latitude is :-" ,lat,"\nlongtitude is:-" ,long)
    print("*********************************************************************")
df_sbd_dc['latitude'] = lat_list
df_sbd_dc['longitude'] = long_list
df_sbd_dc

Jackson TN
latitude is :- 35.6144446 
longtitude is:- -88.8177418
*********************************************************************
Sedalia MO
latitude is :- 38.7098314 
longtitude is:- -93.2295515
*********************************************************************
East Longmeadow MA
latitude is :- 42.064906 
longtitude is:- -72.5129296
*********************************************************************
Ft. Mill SC
latitude is :- 35.0073697 
longtitude is:- -80.9450759
*********************************************************************
Northlake TX
latitude is :- 33.1273432 
longtitude is:- -97.2655726
*********************************************************************
Fontana CA
latitude is :- 34.0922947 
longtitude is:- -117.43433
*********************************************************************
Hagerstown MD
latitude is :- 39.6430455 
longtitude is:- -77.7191081
*********************************************************************
Shannon MS
latitude is :- 34.1162203

,LOC_ID,LOC_NAME,SHP_CITY_PSTL_CD,SHP_CITY_NAME,SHP_STATE,latitude,longitude
0,38,Jackson - DC,38305,Jackson,TN,35.614445,-88.817742
1,120,Sedalia DC,65301,Sedalia,MO,38.709831,-93.229551
2,134,East Longmeadow DC,1028,East Longmeadow,MA,42.064906,-72.512930
3,155,FT. MILL BLACK & DECKER CORP,29715,Ft. Mill,SC,35.007370,-80.945076
4,162,Northlake DC,76262,Northlake,TX,33.127343,-97.265573
5,164,Fontana DC Black & Decker Inc.,92337,Fontana,CA,34.092295,-117.434330
6,166,Hagerstown DC,21740,Hagerstown,MD,39.643045,-77.719108
7,362,Tupelo DC,38868,Shannon,MS,34.116220,-88.711718
8,605,Gallatin DC,37066,Gallatin,TN,36.388303,-86.447590
9,703,Kannapolis DC,28027,Concord,NC,35.409418,-80.580005


In [27]:
df_cust_dc = pd.read_csv("cust_dc.txt")
df_cust_dc['CITY'] = df_cust_dc.index
lat_list = []
long_list = []
# Initialize Nominatim API
geolocator = Nominatim(user_agent="test_cust_dc")

for state, country, city in df_cust_dc.values.tolist():
    loc = geolocator.geocode(city + ', ' + state + ', '+ country)
    lat, long = loc.latitude, loc.longitude
    lat_list.append(lat)
    long_list.append(long)
    print(city, state)
    print("latitude is :-" ,lat,"\nlongtitude is:-" ,long)
    print("*********************************************************************")
df_cust_dc['latitude'] = lat_list
df_cust_dc['longitude'] = long_list
df_cust_dc

HOUSTON  TX
latitude is :- 29.7589382 
longtitude is:- -95.3676974
*********************************************************************
LOCUST GROVE  GA
latitude is :- 33.3459495 
longtitude is:- -84.109083
*********************************************************************
DALLAS  TX
latitude is :- 32.7762719 
longtitude is:- -96.7968559
*********************************************************************
CHEYENNE  WY
latitude is :- 41.139981 
longtitude is:- -104.820246
*********************************************************************
MCDONOUGH  GA
latitude is :- 33.4473361 
longtitude is:- -84.1468616
*********************************************************************
PHOENIX  AZ
latitude is :- 33.4484367 
longtitude is:- -112.074141
*********************************************************************
STATESVILLE  NC
latitude is :- 35.7826363 
longtitude is:- -80.8872959
*********************************************************************
KISSIMMEE  FL
latitude is :- 28.

,DC_Address,COUNTRY,CITY,latitude,longitude
HOUSTON,TX,USA,HOUSTON,29.758938,-95.367697
LOCUST GROVE,GA,USA,LOCUST GROVE,33.345950,-84.109083
DALLAS,TX,USA,DALLAS,32.776272,-96.796856
CHEYENNE,WY,USA,CHEYENNE,41.139981,-104.820246
MCDONOUGH,GA,USA,MCDONOUGH,33.447336,-84.146862
...,...,...,...,...,...
VAN BUREN,OH,USA,VAN BUREN,41.138686,-83.649473
FARGO,ND,USA,FARGO,46.877229,-96.789821
ROCKY VIEW COUNTY,AB,Canada,ROCKY VIEW COUNTY,51.369935,-114.014186
CALGARY,AB,USA,CALGARY,31.617400,-94.119362


In [29]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    hav = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(hav))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'], v['long'], p['lat'], p['long']))

In [33]:
sbd_dc_lat_long_list = []
for lat, long in df_sbd_dc[['latitude', 'longitude']].values.tolist():
    #print(lat, long)
    sbd_dc_lat_long_list.append({'lat' : lat, 'long': long})
sbd_dc_lat_long_list

[{'lat': 35.6144446, 'long': -88.8177418},
 {'lat': 38.7098314, 'long': -93.2295515},
 {'lat': 42.064906, 'long': -72.5129296},
 {'lat': 35.0073697, 'long': -80.9450759},
 {'lat': 33.1273432, 'long': -97.2655726},
 {'lat': 34.0922947, 'long': -117.43433},
 {'lat': 39.6430455, 'long': -77.7191081},
 {'lat': 34.1162203, 'long': -88.7117181},
 {'lat': 36.3883031, 'long': -86.4475898},
 {'lat': 35.4094178, 'long': -80.5800049}]

In [44]:
nearest_sbd_dc_lat_list = []
nearest_sbd_dc_long_list = []
for lat, long in df_cust_dc[['latitude', 'longitude']].values.tolist():
    anchor = {'lat': lat, 'long': long}
    nearest_sbd_dc_lat_list.append(closest(sbd_dc_lat_long_list, anchor)['lat'])
    nearest_sbd_dc_long_list.append(closest(sbd_dc_lat_long_list, anchor)['long'])
    #print(closest(sbd_dc_lat_long_list, anchor))
df_cust_dc['nearest_sbd_dc_lat'] = nearest_sbd_dc_lat_list
df_cust_dc['nearest_sbd_dc_long'] = nearest_sbd_dc_long_list
df_cust_dc.head()

,DC_Address,COUNTRY,CITY,latitude,longitude,nearest_sbd_dc_lat,nearest_sbd_dc_long
HOUSTON,TX,USA,HOUSTON,29.76,-95.37,33.127343,-97.265573
LOCUST GROVE,GA,USA,LOCUST GROVE,33.35,-84.11,35.007370,-80.945076
DALLAS,TX,USA,DALLAS,32.78,-96.80,33.127343,-97.265573
CHEYENNE,WY,USA,CHEYENNE,41.14,-104.82,38.709831,-93.229551
MCDONOUGH,GA,USA,MCDONOUGH,33.45,-84.15,35.007370,-80.945076


In [45]:
df_sbd_dc['latitude'] = df_sbd_dc['latitude'].apply(lambda x : np.round(x, 2))
df_sbd_dc['longitude'] = df_sbd_dc['longitude'].apply(lambda x : np.round(x, 2))

df_cust_dc['latitude'] = df_cust_dc['latitude'].apply(lambda x : np.round(x, 2))
df_cust_dc['longitude'] = df_cust_dc['longitude'].apply(lambda x : np.round(x, 2))

df_cust_dc['nearest_sbd_dc_lat'] = df_cust_dc['nearest_sbd_dc_lat'].apply(lambda x : np.round(x, 2))
df_cust_dc['nearest_sbd_dc_long'] = df_cust_dc['nearest_sbd_dc_long'].apply(lambda x : np.round(x, 2))

In [46]:
df_cust_dc.head()

,DC_Address,COUNTRY,CITY,latitude,longitude,nearest_sbd_dc_lat,nearest_sbd_dc_long
HOUSTON,TX,USA,HOUSTON,29.76,-95.37,33.13,-97.27
LOCUST GROVE,GA,USA,LOCUST GROVE,33.35,-84.11,35.01,-80.95
DALLAS,TX,USA,DALLAS,32.78,-96.80,33.13,-97.27
CHEYENNE,WY,USA,CHEYENNE,41.14,-104.82,38.71,-93.23
MCDONOUGH,GA,USA,MCDONOUGH,33.45,-84.15,35.01,-80.95


In [47]:
df_sbd_dc[(df_sbd_dc.latitude == 33.13) & (df_sbd_dc.longitude == -97.27)]

,LOC_ID,LOC_NAME,SHP_CITY_PSTL_CD,SHP_CITY_NAME,SHP_STATE,latitude,longitude
4,162,Northlake DC,76262,Northlake,TX,33.13,-97.27


In [63]:
df_cust_dc_sbd_dc = pd.merge(df_cust_dc, df_sbd_dc, how = 'inner', left_on=['nearest_sbd_dc_lat', 'nearest_sbd_dc_long'], right_on = ['latitude', 'longitude'])
df_cust_dc_sbd_dc.columns = [col.upper() for col in df_cust_dc_sbd_dc.columns]
df_cust_dc_sbd_dc = pd.merge(df_cust_dc, df_sbd_dc, how = 'inner', left_on=['nearest_sbd_dc_lat', 'nearest_sbd_dc_long'], right_on = ['latitude', 'longitude'])
df_cust_dc_sbd_dc.columns = [col.upper() for col in df_cust_dc_sbd_dc.columns]
df_cust_dc_sbd_dc.columns = ['CUST_DC_STATE', 'CUST_DC_COUNTRY', 'CUST_DC_CITY', 'CUST_DC_LATITUDE', 'CUST_DC_LONGITUDE',
       'NEAREST_SBD_DC_LAT', 'NEAREST_SBD_DC_LONG', 'SBD_DC_LOC_ID', 'SBD_DC_LOC_NAME',
       'SBD_DC_SHP_CITY_PSTL_CD', 'SBD_DC_SHP_CITY_NAME', 'SBD_DC_SHP_STATE', 'LATITUDE_Y',
       'LONGITUDE_Y']
df_cust_dc_sbd_dc[['CUST_DC_CITY', 'CUST_DC_STATE', 'CUST_DC_COUNTRY', 'CUST_DC_LATITUDE', 'CUST_DC_LONGITUDE',
       'NEAREST_SBD_DC_LAT', 'NEAREST_SBD_DC_LONG', 'SBD_DC_LOC_ID', 'SBD_DC_LOC_NAME',
       'SBD_DC_SHP_CITY_NAME', 'SBD_DC_SHP_STATE',  'SBD_DC_SHP_CITY_PSTL_CD', ]]

,CUST_DC_CITY,CUST_DC_STATE,CUST_DC_COUNTRY,CUST_DC_LATITUDE,CUST_DC_LONGITUDE,NEAREST_SBD_DC_LAT,NEAREST_SBD_DC_LONG,SBD_DC_LOC_ID,SBD_DC_LOC_NAME,SBD_DC_SHP_CITY_NAME,SBD_DC_SHP_STATE,SBD_DC_SHP_CITY_PSTL_CD
0,HOUSTON,TX,USA,29.76,-95.37,33.13,-97.27,162,Northlake DC,Northlake,TX,76262
1,DALLAS,TX,USA,32.78,-96.80,33.13,-97.27,162,Northlake DC,Northlake,TX,76262
2,DENTON,TX,USA,33.18,-97.14,33.13,-97.27,162,Northlake DC,Northlake,TX,76262
3,MOUNT VERNON,TX,USA,33.19,-95.22,33.13,-97.27,162,Northlake DC,Northlake,TX,76262
4,ADA,OK,USA,34.77,-96.68,33.13,-97.27,162,Northlake DC,Northlake,TX,76262
...,...,...,...,...,...,...,...,...,...,...,...,...
57,EAST POINT,GA,USA,33.68,-84.44,36.39,-86.45,605,Gallatin DC,Gallatin,TN,37066
58,PLAINFIELD,IN,USA,39.70,-86.40,36.39,-86.45,605,Gallatin DC,Gallatin,TN,37066
59,ADAIRSVILLE,GA,USA,34.37,-84.93,36.39,-86.45,605,Gallatin DC,Gallatin,TN,37066
60,NORTH VERNON,IN,USA,39.01,-85.62,36.39,-86.45,605,Gallatin DC,Gallatin,TN,37066


In [64]:
df_cust_dc_sbd_dc.to_csv("nearest_sbd_dc.csv", index = False)